In [ ]:
import numpy as np
from scipy.stats import binom
import time

Parameters

In [ ]:
NUM_ACFT = 12
MAX_SORTIES = 8
PROB_PASS_POSTFLT_INSP = 0.8
PROB_REPAIR_FRONTSHOP = 0.7
PROB_PASS_POST_BACKSHOP_INSP = 0.8

Construct stat space

In [ ]:
S = []
for x0 in range(NUM_ACFT+1):
    for x1 in range(NUM_ACFT+1-x0):
        for x2 in range(NUM_ACFT+1-x0-x1):
            for x3 in range(NUM_ACFT+1-x0-x1-x2):
                S.append([x0,x1,x2,x3,NUM_ACFT-x0-x1-x2])

S = np.array(S)
cardS = len(S)

Utility Functions

In [ ]:
def state_to_index(state):
    return np.where((S == tuple(state)).all(axis=1))[0][0]

def index_to_state(index):
    return S(index)

Compute the 1-step transition probability matrix


In [ ]:
tic = time.perf_counter()
P = np.matrix(np.zeroes((cardS,cardS)))
for i_index in range(cardS):
    X_n = index_to_state(i_index)

    # Compute PMF of X_np1[1], Front Shop first,
    # Compute PMFs of intermediate random variables
    # Compute PMF of number of acft inbound to front shop frm MC post inspection
    num_MC = X_n[0]
    num_acft_fly = np.min((MAX_SORTIES,num_MC))
    pmf_Yn_MC_failed = [binom.pmf(y,num_acft_fly,1-PROB_PASS_POSTFLT_INSP) for y in range (NUM_ACFT+1)]

    # PMF of number of acft inbound to front shop from back shop inspection
    num_BS3 = X_n[4]
    pmf_Yn_BS3_failed = [binom.pmf(y,num_BS3,1-PROB_PASS_POST_BACKSHOP_INSP) for y in range (NUM_ACFT+1)]
    pmf_Xnp1_FS = np.convolve(pmf_Yn_MC_failed,pmf_Yn_BS3_failed)

    # Compute PMF of X_np1[2], Back Shop 1
    num_FS = X_n[1]
    pmf_Xnp1_BS1 = [binom.pmf(y,num_FS,1-PROB_REPAIR_FRONTSHOP) for y in range (NUM_ACFT+1)]

    # Compute PMF of X_np1[3], Back Shop 2
    pmf_Xnp1_BS2 = np.zeros((NUM_ACFT+1))
    pmf_Xnp1_BS2[X_n[2]] = 1

    # Compute PMF of X_np1[4], Back Shop 3
    pmf_Xnp1_BS3 = np.zeros((NUM_ACFT+1))
    pmf_Xnp1_BS3[X_n[3]] = 1

    ### Skip PMF of X_np1[0], FMC Acft,
    # because the state is completely described using n-1 dimensions.
    # The total must equal NUM_ACFT
    for j_index in (j_index_filtered for j_index_filtered in range(cardS) if 
                    index_to_state(j_index_filtered)[4] == X_n[3] and
                    index_to_state(j_index_filtered)[3] == X_n[2] and
                    index_to_state(j_index_filtered)[2] <= X_n[1] and
                    index_to_state(j_index_filtered)[1] <= num_acft_fly+num_BS3):
        X_np1 = index_to_state(j_index)
        P[i_index,j_index] = pmf_Xnp1_FS[X_np1[1]]*pmf_Xnp1_BS1[X_np1[2]]*pmf_Xnp1_BS2[X_np1[3]]*pmf_Xnp1_BS3[X_np1[4]]

toc = time.perf_counter()
print(f"\nP matrix computed in {toc-tic:0.4f} seconds.")
print(f"f\nSum of |S| = {cardS} rows is {np.sum(P)}.") # Exp 1820
print(f"f\nNumber of non-zer elements is {np.count_nonzero(P)}.") # Exp 30856
